# Crystallization solvent extraction with NLP


Installing required packages.

In [ ]:
!pip install chemdataextractor # for NLP
!pip install fuzzywuzzy        # for SI download
!pip install selenium          # for SI download
!cde data download            # download the data for chemdataextractor

     |████████████████████████████████| 182 kB 7.9 MB/s 
     |████████████████████████████████| 965 kB 51.9 MB/s 
     |████████████████████████████████| 371 kB 45.2 MB/s 
     |████████████████████████████████| 5.6 MB 5.2 MB/s 
     |████████████████████████████████| 3.6 MB 43.0 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=867631 sha256=ac3a59e665a32f11fb911110a3f904cc648f75870d20c3a0e4131fea004b27df
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_crf-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_crf-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_crf_chemdner_cemp-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_crf_chemdner_cemp-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_dict_cs-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_dict_cs-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_dict-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_dict-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/clusters_chem1500-1.0.pickle to /root/.local/share/ChemDataExtractor/models/clusters_chem1500-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/pos_ap_genia_nocluster-1.0.pick

     |████████████████████████████████| 968 kB 9.1 MB/s 
     |████████████████████████████████| 359 kB 46.6 MB/s 
     |████████████████████████████████| 138 kB 58.5 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 58 kB 4.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Getting the crystallization data extraction code from our GitHub repository

In [ ]:
!git clone https://github.com/caer200/solvent_nlp

Cloning into 'solvent_nlp'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 8), reused 15 (delta 4), pack-reused 0
Unpacking objects: 100% (25/25), done.


We first import the required code

In [ ]:
from solvent_nlp.webscraping import *        # this code is for webscraping the PDF SI
from solvent_nlp.solvent_parser import *     # this code parses the solvent from text

The function below returns a the compound names and solvent used for crystallization 
from the parsed records. 

In [ ]:
def get_compounds_with_solvent(records):
    data = []
    for compound in records:
        if compound.get("crystallization"):
            data.append(compound)
    return data

Let us fetch a SI using the webscraping code

In [ ]:
saveloc = "."  # location to save the PDF
doi = "10.1039/C4OB01574F"   

sc = SIScraper(doi=doi, saveloc=saveloc)
sc.retrieveSI()

/content/solvent_nlp/webscraping.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/solvent_nlp/webscraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(self.html)


We use chemdataextractor (https://chemdataextractor.org) to do the processsing followed
by tokenization. The rule-based parsing of crystallization solvent is described in the solvent_parser.py.
We use the CompundParser that is built-in chemdataextractor and the SolventParser that we develop to extract
the crystallization solvent.

In [ ]:
doc = Document.from_file(sc.filename)  # loading the PDF
records = doc.records.serialize()  # fetching the results from NLP as a python dict
get_compounds_with_solvent(records)

[{'crystallization': [{'solvent': 'DCM / PE'}], 'names': ['Li']},
 {'crystallization': [{'solvent': 'DCM / PE'}], 'names': ['S1-5']},
 {'crystallization': [{'solvent': 'DCM and methanol'}],
  'labels': ['3i'],
  'names': ['(S)-(L)-menthyl [(R)-1-(4-methoxylphenyl)-1-hydroxyethyl] phenylphosphinate']},
 {'crystallization': [{'solvent': 'DCM and methanol'}],
  'labels': ['3j'],
  'names': ['(S)-(L)-menthyl [(R)-1-biphenyl-1-hydroxyethyl] phenylphosphinate']},
 {'crystallization': [{'solvent': 'DCM and methanol'}],
  'labels': ['3u'],
  'names': ['(S)-(L)-menthyl [ (R)-(2-hydroxy-4-methyl pentan-2-yl ) ] phenylphosphinate']},
 {'crystallization': [{'solvent': 'DCM'}],
  'labels': ['3db'],
  'names': ['phenylphosphinate',
   'Cyclohexyl [1-hydroxy-1-(4-bromophenyl)ethyl]phenylphosphinate']},
 {'crystallization': [{'solvent': 'DCM and methanol'}],
  'labels': ['3eb'],
  'names': ['tert-Butyl [ 1-hydroxy-1-(4-bromophenyl) ethyl]phenylphosphinate',
   'tert-Butyl [1-hydroxy-1-(4-bromophenyl)e